In [1]:
from pathlib import Path

import ee

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
print('Testing GEE access...', ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

Testing access... NASADEM: NASA NASADEM Digital Elevation 30m


In [3]:
! geeadd -h

usage: geeadd [-h]
              {readme,quota,app2script,search,ee_report,assetsize,tasks,cancel,copy,move,access,delete,delete_metadata}
              ...

Google Earth Engine Batch Asset Manager with Addons

positional arguments:
  {readme,quota,app2script,search,ee_report,assetsize,tasks,cancel,copy,move,access,delete,delete_metadata}
    readme              Go the web based porder readme page
    quota               Print Earth Engine total quota and used quota
    app2script          Get underlying script for public Google earthengine
                        app
    search              Search public GEE catalog using keywords
    ee_report           Prints a detailed report of all Earth Engine Assets
                        includes Asset Type, Path,Number of
                        Assets,size(MB),unit,owner,readers,writers
    assetsize           Prints any asset size (folders,collections,images or
                        tables) in Human Readable form & Number of assets
      

In [9]:
! geeadd quota


Root assets path: users/terencebarrett
Used 22.08 GB of 250 GB
Used 4,646 assets of 50,000 total


In [25]:
! geeadd ee_report -h

usage: geeadd ee_report [-h] --outfile OUTFILE

options:
  -h, --help         show this help message and exit

Required named arguments.:
  --outfile OUTFILE  This it the location of your report csv file


In [10]:
# geeadd's ee_report functionality is not currently working
report_file = Path.cwd() / 'ee_report.csv'
print('Generating a report on Asset holdings to: ', report_file)
! geeadd ee_report --outfile report_file

Generating a report on Asset holdings to:  /home/terencebarrett/PycharmProjects/gee_asset_management/ee_report.csv
[D 220824 13:59:07 ee_report:144] This might take sometime. Go grab some tea.....
[D 220824 13:59:09 ee_report:165] Processing your root folder: projects/earthengine-legacy/assets/users/terencebarrett
[D 220824 13:59:12 ee_report:167] Processing a total of: 3 folders 0 collections 0 images 5 tables
    
[I 220824 13:59:12 ee_report:177] Processing Folder projects/earthengine-legacy/assets/users/terencebarrett/CoastXRayPython
usage: earthengine [-h] [--ee_config EE_CONFIG]
                   [--service_account_file SERVICE_ACCOUNT_FILE]
                   [--project PROJECT_OVERRIDE]
                   {authenticate,acl,asset,cp,create,ls,licenses,du,mv,model,rm,set_project,task,unset_project,upload,upload_manifest,upload_table_manifest}
                   ...
earthengine: error: unrecognized arguments: --no-use_cloud_api
Traceback (most recent call last):
  File "/home/ter

In [11]:
!geeadd copy -h

usage: geeadd copy [-h] [--initial INITIAL] [--final FINAL]

options:
  -h, --help         show this help message and exit

Required named arguments.:
  --initial INITIAL  Existing path of assets
  --final FINAL      New path for assets


In [5]:
# Optional stepping-stone test:
#  Recursively copy a Folder of Assets from Legacy Assets to Cloud Assets,
#   when both are owned by this user
!geeadd copy --initial users/terencebarrett/Grid --final projects/ee-terencebarrett/assets/Grid

Total folders: 1
Folder does not exist: Creating projects/ee-terencebarrett/assets/Grid
Copying table to projects/ee-terencebarrett/assets/Grid/ISEA3H_11_Maine_Jan2022_WGS84
Copying table to projects/ee-terencebarrett/assets/Grid/ISEA3H_11_Maine_WGS84
Copying table to projects/ee-terencebarrett/assets/Grid/ISEA3H_12_Maine_WGS84
Copying table to projects/ee-terencebarrett/assets/Grid/ISEA3H_12_Maine_WGS84_cell_405303
Copying table to projects/ee-terencebarrett/assets/Grid/dggrid_11_me_nh_cusp_jan_2022_wgs84_utm_19N



In [3]:
# Optional stepping-stone test:
#  Copy another user's single Cloud Asset (to which this user
#   has Reader access) to an existing folder in this user's Cloud Assets
!geeadd copy --initial projects/ee-nearview/assets/Beals_100m_16Aug2021 --final projects/ee-terencebarrett/assets/Testcopy/Beals_100m_16Aug2021_3

Copying image to projects/ee-terencebarrett/assets/Testcopy/Beals_100m_16Aug2021_3


In [6]:
# Recursively copy a folder of Assets from this user's Assets to
#  another user's Cloud Assets folder to which this user has Writer access;
#  this user must also have been added as a Principal in the Editor Role to
#  the destination Cloud Assets space
# !geeadd copy --initial users/terencebarrett/Grid --final projects/ee-nearview/assets/CXR/Grid
!geeadd copy --initial users/terencebarrett/CoastXRayPython --final projects/ee-nearview/assets/CXR/CoastXRayPython

Total folders: 21
Folder does not exist: Creating projects/ee-nearview/assets/CXR/CoastXRayPython

Folder does not exist: Creating projects/ee-nearview/assets/CXR/CoastXRayPython/GridCellSummary
Copying table to projects/ee-nearview/assets/CXR/CoastXRayPython/GridCellSummary/Maine_1C_Grid11_noWaterClean_2015-09-01_2021-11-30_collection
Copying table to projects/ee-nearview/assets/CXR/CoastXRayPython/GridCellSummary/Maine_1C_Grid11_noWaterClean_utm19n_2015-09-01_2021-11-30_collection
Copying table to projects/ee-nearview/assets/CXR/CoastXRayPython/GridCellSummary/Maine_2015-09-01_2021-11-30_collection
Copying table to projects/ee-nearview/assets/CXR/CoastXRayPython/GridCellSummary/Maine_2017-01-01_2018-12-31_collection
Copying table to projects/ee-nearview/assets/CXR/CoastXRayPython/GridCellSummary/Maine_2018-01-01_2018-12-31_collection
Copying table to projects/ee-nearview/assets/CXR/CoastXRayPython/GridCellSummary/Maine_2019-01-01_2020-12-31_collection
Copying table to projects/ee-nea

In [4]:
# Alternate -- non-recursive, non-geeadd -- method to copy a folder and its
#  assets from this user's Legacy Assets to another user's Cloud Assets folder
#  to which this user has the same access as in the previous method
src_folder = "projects/earthengine-legacy/assets/users/terencebarrett/Grid"
dest_folder = "projects/ee-nearview/assets/CXR"

# Get all assets in the folder
assets = ee.data.listAssets({'parent': src_folder})

# Create new folder
new_folder = dest_folder + '/' + src_folder.split('/')[-1]
print(f'Creating new folder: {new_folder}')
ee.data.createAsset({'type': 'Folder'}, new_folder)

# Loop through assets and copy them one by one to the new destination
for asset in assets['assets']:
    # Construct destination path
    new_asset = new_folder + '/' + asset['id'].split('/')[-1]

    # Copy to destination
    print(f'Copying {asset["id"]} to {new_asset}')
    ee.data.copyAsset(asset['id'], new_asset, True)

Creating new folder: projects/ee-nearview/assets/CXR/Grid
Copying users/terencebarrett/Grid/ISEA3H_11_Maine_Jan2022_WGS84 to projects/ee-nearview/assets/CXR/Grid/ISEA3H_11_Maine_Jan2022_WGS84
Copying users/terencebarrett/Grid/ISEA3H_11_Maine_WGS84 to projects/ee-nearview/assets/CXR/Grid/ISEA3H_11_Maine_WGS84
Copying users/terencebarrett/Grid/ISEA3H_12_Maine_WGS84 to projects/ee-nearview/assets/CXR/Grid/ISEA3H_12_Maine_WGS84
Copying users/terencebarrett/Grid/ISEA3H_12_Maine_WGS84_cell_405303 to projects/ee-nearview/assets/CXR/Grid/ISEA3H_12_Maine_WGS84_cell_405303
Copying users/terencebarrett/Grid/dggrid_11_me_nh_cusp_jan_2022_wgs84_utm_19N to projects/ee-nearview/assets/CXR/Grid/dggrid_11_me_nh_cusp_jan_2022_wgs84_utm_19N
